# ETL for Reddit Data

This notebook extracts comments from the hottest posts in specified subreddits.

Example of simple data exploration is shown with wordcloud.

### 1. Import dependencies

In [1]:
import pandas as pd
import re

## reddit API wrapper (https://praw.readthedocs.io)
import praw


### 2. Set up reddit instance
Requires Reddit API keys `client_id`, `client_secret`, and `user_agent` to be declared in the separate file "secrets.py".

In [2]:
from secrets import *

reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=CLIENT_SECRET,
                     user_agent=USER_AGENT)


### 3. Easy wrappers for praw

Reddit is organized into a forest: The roots are called *subreddits*, to which users contribute *submissions*. Each submission has its own comment forest, a collection of comments on the submission itself or on other comments. At the submission level and below, Reddit sorts each submission or comment according to *hotness*, a measure of popularity. This organization is intuitive from the interactive frontend, but the API is a little confusing. Praw is the dominant Python wrapper for the Reddit API.

Praw defines a `Reddit` instance, providing access to the site. From there we may request `Submissions` from any subreddit (or we can ask for 'all' to aggregate across all subreddits). These submissions come with their attached comment forests in the `comments` attribute. We can get a flattened list of all comments from the `list` method of `comments`.

##### Pruning the Comment Forests
Here's where things get tricky: Reddit prunes the comment forests, leaving `MoreComments` placeholders. We may request these pruned comments from Reddit using the `replace_more` method of the praw `Comment` object, up to *limit* comments. We may also set a *threshold* for how many children (including `MoreComments` placeholders) a comment must have to be included.

This pruning is performed because, in practice, it is often the case that only the top few levels of comments are interesting, with the rest devolving into off-topic conversation. Praw does not offer an easy way to specify the maximum depth of the comment trees to be returned. The function `get_comments` implements a breadth-first search to enforce a hard maximum depth. When no maximum depth is specified, the only pruning done is by Reddit's `MoreComments` feature, and the breadth first search is equivalent to the list method of a praw `Submission`'s comment forest.

Of course, flattening and pruning the tree loses information: it would be impossible to read a thread and understand the conversation. However, storing the comments in a tidy structure like a DataFrame makes analysis of the submission corpora much easier.

##### Transforming reddit markdown
The function `to_dataframe` strips leading/trailing whitespace and all newline characters. It also attempts to remove markdown metacharacters, as detailed [here](https://www.reddit.com/r/reddit.com/comments/6ewgt/reddit_markdown_primer_or_how_do_you_do_all_that/c03nik6/), to mixed success. In particular, it does not gracefully handle quotes/superscripts (which both use a carat >); carats are simply removed, sometimes joining superscript to its base. The function also simply leaves links alone, with format \[text\](link "tooltip text").

In [3]:
def get_comments(submission, depth=None, limit=32, threshold=0):
    '''Flatten comment forest up to specified depth after replacing 'MoreComments' instances.
    
    Keyword arguments:
    submission -- submission instance from praw
    depth      -- maximum traversal depth of comment tree; None returns whole tree (default None)
    limit      -- number of MoreComments instances to replace (default 32)
    threshold  -- minimum number of children required to expand MoreComments (default 0)
    '''
    submission.comments.replace_more(limit=limit, threshold=threshold)
    
    if depth == None:
        return submission.comments.list()
    
    comments = []
    level = 0
    sentinel = ['placeholder']
    comment_queue = submission.comments[:]
    comment_queue.extend(sentinel)
    while comment_queue and level < depth:
        comment = comment_queue.pop(0)
        if [comment] == sentinel:
            level = level + 1
            comment_queue.extend(sentinel)
            continue
        comments.append(comment)
        comment_queue.extend(comment.replies)
    return comments

def to_dataframe(comments, columns=['author', 'created_utc']):
    '''Transform comments list into pandas dataframe. Strip newlines and leading/trailing whitespace.
    
    Keyword arguments:
    comments -- list of praw Comment instances
    columns  -- list of column names corresponding to praw Comment attributes (default ['author', 'created_utc'])
    '''
    data = pd.DataFrame(data=[comment.body for comment in comments],columns=['body'])
    for col in columns:
        data[col] = np.array([getattr(comment,col) for comment in comments])
        
    for index, row in data.iterrows():
        line = row.body
        line = re.sub('[\\n]', ' ', line)
        line = re.sub('~~|\*|_|#|\`|\>', '', line)
        line = line.strip()
        data.loc[index,'body'] = line
    return data

def load_data(reddit, subreddit='all', limit_subs=10, depth=None, limit_coms=32, threshold=0):
    '''Extract, transform, and load comments into dataframe.
    
    Keyword arguments:
    reddit     -- reddit instance from praw
    subreddit  -- name of subreddit (default 'all')
    limit_subs -- number of submissions to extract
    depth      -- maximum traversal depth of comment tree; None returns whole tree (default None)
    limit_coms -- number of MoreComments instances to replace (default 32)
    threshold  -- minimum number of children required to expand MoreComments (default 0)
    '''
    submissions = [submission for submission in reddit.subreddit(subreddit).hot(limit=limit_subs)]
    comments = []
    for submission in submissions:
        comments.extend(get_comments(submission, depth=depth, limit=limit_coms, threshold=threshold))
    data = to_dataframe(comments)
    return data, submissions, comments


### Example: load comments from hot submissions in /r/AskReddit

In [4]:
data, submissions, comments = load_data(reddit, subreddit='AskReddit', limit_subs=5, depth=3)
data.head()


NameError: name 'np' is not defined

### Create wordcloud from all loaded comments

In [ ]:
## Andy Mueller's wordcloud (https://github.com/amueller/word_cloud)
from wordcloud import WordCloud
import matplotlib.pyplot as plt

## Join all comments into one string
text = ' '.join(data['body'])

## Generate the wordcloud
wordcloud = WordCloud().generate(text)

## Display wordcloud with matplotlib
_ = plt.imshow(wordcloud, interpolation='bilinear')
_ = plt.axis("off")
plt.show()
